In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TitanicDataProcessing") \
    .enableHiveSupport() \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df = spark.read.csv("s3://titanic1m/huge_1M_titanic.csv", header=True, inferSchema=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|              Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------------------+-----+--------+
|       1310|       1|     1|Name1310, Miss. S...|female|NULL|    0|    0|SOTON/O2 3101272| 76.76016504643573| NULL|       C|
|       1311|       0|     3|Name1311, Col. Su...|  male|29.0|    0|    0|          223596|10.193096706320182| NULL|       S|
|       1312|       0|     3|Name1312, Mr. Sur...|  male|20.0|    0|    0|           54636| 12.02941641147422|  C83|       C|
|       1313|       0|     3|Name1313, Mr. Sur...|  male|27.0|    0|    0|        PC 17760|13.429447862759872| NULL|       S|
|       1314|       0|     3|Name1314, Mr. Sur...|  male|32.0|    0|    0|          364512| 4.840769450167068|  E33|  

In [23]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000000

In [24]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

In [25]:
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+------------------+------------------+--------------------+-------+------------------+-----------------+------------------+------------------+-----------------+------+--------+
|summary|      PassengerId|          Survived|            Pclass|                Name|    Sex|               Age|            SibSp|             Parch|            Ticket|             Fare| Cabin|Embarked|
+-------+-----------------+------------------+------------------+--------------------+-------+------------------+-----------------+------------------+------------------+-----------------+------+--------+
|  count|          1000000|           1000000|           1000000|             1000000|1000000|            801400|          1000000|           1000000|           1000000|          1000000|229805|  997760|
|   mean|         501309.5|          0.381681|           2.31125|                NULL|   NULL| 29.28033940603943|         0.511101|          0.360037|259498.87949019438|33.461097240381

In [26]:
from pyspark.sql.functions import col, isnan, when, count, isnull, mean

def null_value_count(df):
    null_counts = []
    for c in df.columns:
        if df.schema[c].dataType.simpleString() in ('double', 'float'):

            null_counts.append(count(when(col(c).isNull() | isnan(c), c)).alias(c))
        else:
            null_counts.append(count(when(isnull(c), c)).alias(c))
    
    return df.select(null_counts).show()

null_value_count(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------+----+---+------+-----+-----+------+----+------+--------+
|PassengerId|Survived|Pclass|Name|Sex|   Age|SibSp|Parch|Ticket|Fare| Cabin|Embarked|
+-----------+--------+------+----+---+------+-----+-----+------+----+------+--------+
|          0|       0|     0|   0|  0|198600|    0|    0|     0|   0|770195|    2240|
+-----------+--------+------+----+---+------+-----+-----+------+----+------+--------+

In [27]:
mean_age = df.select(mean(col("Age"))).collect()[0][0]
df = df.withColumn("Age", when(isnull(col("Age")), mean_age).otherwise(col("Age")))

df = df.withColumn("HasCabin", when(isnull(col("Cabin")), False).otherwise(True))

most_common_port = df.groupBy("Embarked").count().orderBy("count", ascending=False).first()[0]
df = df.withColumn("Embarked", when(isnull(col("Embarked")), most_common_port).otherwise(col("Embarked")))

null_value_count(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------+----+---+---+-----+-----+------+----+------+--------+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare| Cabin|Embarked|HasCabin|
+-----------+--------+------+----+---+---+-----+-----+------+----+------+--------+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|770195|       0|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+------+--------+--------+

In [28]:
df = df.drop('PassengerId', 'Name', 'Ticket', 'Cabin')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+-----------------+-----+-----+------------------+--------+--------+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|              Fare|Embarked|HasCabin|
+--------+------+------+-----------------+-----+-----+------------------+--------+--------+
|       1|     1|female|29.28033940603943|    0|    0| 76.76016504643573|       C|   false|
|       0|     3|  male|             29.0|    0|    0|10.193096706320182|       S|   false|
|       0|     3|  male|             20.0|    0|    0| 12.02941641147422|       C|    true|
|       0|     3|  male|             27.0|    0|    0|13.429447862759872|       S|   false|
|       0|     3|  male|             32.0|    0|    0| 4.840769450167068|       C|    true|
+--------+------+------+-----------------+-----+-----+------------------+--------+--------+
only showing top 5 rows

In [38]:
output_path = "s3://titanic1m/titanic_data/"
df.write.mode("overwrite").parquet(output_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'

